In [1]:
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Point
import re

def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [2]:
hospitals = pd.read_csv('RS_iGEM.csv')
hospitals.head()

,No,Name,Address,Class,District,Longitude,Latitude,Pop,Envi_Fact
0,1,RSK Bedah dan Penyakit Dalam BIMC,"Jl. By Pass Ngurah Rai No.100 X, Kuta - Badung",NaN,Badung,115.2055221,-8.677749542,4.751959,0.476117
1,2,Rumkit Tk IV Dr. Bratanata Unang Jambi,"Jl. Raden Mattaher 33,Jambi",4,NaN,103.6185346,-1.591743008,3.346735,0.861326
2,3,RSAB Sari Asih,Jl. Imam Bonjol Karawaci No 38 TNG,B,Kota Tangerang,106.626283,-6.179226039,6.780755,0.252343
3,4,RSU Dr Adjidarmo,Jl. Iko Jatmika 1 Rangkasbitung,B,Lebak,106.2831106,-6.369179285,0.069562,0.633958
4,5,RS Usada Insani,Jl. KH.HASYIM ASHARI NO.24 Tangerang,B,Kota Tangerang,106.6477491,-6.1839839,0.629374,0.150169


In [9]:
# Data Cleansing

list_of_points, errors = [], []
for i in range(len(hospitals)):
    lng = float(hospitals.iloc[i].Longitude.replace(',-', '').replace(',', ''))
    lat = float(hospitals.iloc[i].Latitude.replace(',', ''))
    if (95.31644  > lng > 140.71813) or (-10.1718 > lat > 5.88969):
        errors.append(Point(lng, lat))
    else:
        list_of_points.append(Point(lng, lat))

a,b = len(hospitals), len(list_of_points)
if a == b:
    print("Your Data Ready to Go !")
else:
    print("There's still problem mate !")

Your Data Ready to Go !


In [29]:
# CSV Data to GeoDataFrame

hospitals['geometry'] = list_of_points
# hospitals.head()

hospitals = gpd.GeoDataFrame(hospitals)
hospitals.crs = {'init' : 'epsg:4326'}
hospitals.crs

c:\Users\sn\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [31]:
# Saved as shp file (forget about the 2 cells above, use shp instead)
hospitals.to_file("mosquhospital.shp", driver='ESRI Shapefile')

In [42]:
gdf = gpd.read_file('mosquhospital.shp')
gdf.head()

,No,Name,Address,Class,District,Longitude,Latitude,Pop,Envi_Fact,geometry
0,1,RSK Bedah dan Penyakit Dalam BIMC,"Jl. By Pass Ngurah Rai No.100 X, Kuta - Badung",None,Badung,115.2055221,-8.677749542,4.751959,0.476117,POINT (115.20552 -8.67775)
1,2,Rumkit Tk IV Dr. Bratanata Unang Jambi,"Jl. Raden Mattaher 33,Jambi",4,None,103.6185346,-1.591743008,3.346735,0.861326,POINT (103.61853 -1.59174)
2,3,RSAB Sari Asih,Jl. Imam Bonjol Karawaci No 38 TNG,B,Kota Tangerang,106.626283,-6.179226039,6.780755,0.252343,POINT (106.62628 -6.17923)
3,4,RSU Dr Adjidarmo,Jl. Iko Jatmika 1 Rangkasbitung,B,Lebak,106.2831106,-6.369179285,0.069562,0.633958,POINT (106.28311 -6.36918)
4,5,RS Usada Insani,Jl. KH.HASYIM ASHARI NO.24 Tangerang,B,Kota Tangerang,106.6477491,-6.1839839,0.629374,0.150169,POINT (106.64775 -6.18398)


In [43]:
# Normalize data for not-so-bias visualization
norm_pop, norm_envi, max_pop, max_envidx = [], [], max(gdf.Pop), max(gdf.Envi_Fact)
for i in range(len(gdf)):
    norm_pop.append(gdf.iloc[i].Pop / max_pop)
    norm_envi.append(gdf.iloc[i].Envi_Fact / max_envidx)

gdf['Norm_Pop'] = norm_pop
gdf['Norm_Env'] = norm_envi
gdf.head()
# print(norm_pop[:5])
# print(norm_envi[:5])
# print("Jumlah normalisasi populasi ada {} \nJumlah normalisasi index ada {}".format(len(norm_pop), len(norm_envi)))

,No,Name,Address,Class,District,Longitude,Latitude,Pop,Envi_Fact,geometry,Norm_Pop,Norm_Env
0,1,RSK Bedah dan Penyakit Dalam BIMC,"Jl. By Pass Ngurah Rai No.100 X, Kuta - Badung",None,Badung,115.2055221,-8.677749542,4.751959,0.476117,POINT (115.20552 -8.67775),0.409940,0.482668
1,2,Rumkit Tk IV Dr. Bratanata Unang Jambi,"Jl. Raden Mattaher 33,Jambi",4,None,103.6185346,-1.591743008,3.346735,0.861326,POINT (103.61853 -1.59174),0.288715,0.873178
2,3,RSAB Sari Asih,Jl. Imam Bonjol Karawaci No 38 TNG,B,Kota Tangerang,106.626283,-6.179226039,6.780755,0.252343,POINT (106.62628 -6.17923),0.584959,0.255815
3,4,RSU Dr Adjidarmo,Jl. Iko Jatmika 1 Rangkasbitung,B,Lebak,106.2831106,-6.369179285,0.069562,0.633958,POINT (106.28311 -6.36918),0.006001,0.642681
4,5,RS Usada Insani,Jl. KH.HASYIM ASHARI NO.24 Tangerang,B,Kota Tangerang,106.6477491,-6.1839839,0.629374,0.150169,POINT (106.64775 -6.18398),0.054295,0.152235


In [44]:
# Saved as shp file (updated shp, base shp also saved not rewriting it just for backup)
gdf.to_file("norm_mosquital.shp", driver='ESRI Shapefile')